In [ ]:
import yaml
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import sqlalchemy

In [ ]:
with open('credentials.yaml', 'r') as f:
    credentials = yaml.safe_load(f)

In [ ]:
class RDSDatabaseConnector:
    def __init__(self, credentials):
        self.credentials = credentials

    def _create_engine(self):
        engine = create_engine(f"postgresql+psycopg2://{self.credentials['RDS_USER']}:{self.credentials['RDS_PASSWORD']}@{self.credentials['RDS_HOST']}:{self.credentials['RDS_PORT']}/{self.credentials['RDS_DATABASE']}")
        return engine
    
    def initialise_engine(self):
        self.engine = self._create_engine()

    def data_extraction(self, table_name='loan_payments'):
        query = f"SELECT * FROM loan_payments;"
        data = pd.read_sql(query, self.engine)
        return data
    
    def save_to_file(self, data, file_path='loan_payments_data.csv'):
        data.to_csv(file_path, index=False)

    def load_loan_data(self, file_path='loan_payments_data.csv'):
        try:
            return pd.read_csv(file_path)
        except FileNotFoundError:
            print(f"Error: File '{file_path}' not found.")
            return None


In [ ]:
loan_data = RDSDatabaseConnector(credentials)
loan_data._create_engine()
loan_data.initialise_engine()
loan_data.data_extraction()
loan_data.data_extraction('loan_payments')
loan_data.save_to_file(data=loan_data.data_extraction('loan_payments'), file_path='loan_payments_data.csv')
loaded_data = loan_data.load_loan_data('loan_payments_data.csv')

In [ ]:
loans_df = pd.read_csv('loan_payments_data.csv', index_col='id')
loans_df.tail(10).describe()


In [ ]:
loans_df['term'] = loans_df['term'].str.replace('months', '', regex=True)
loans_df['employment_length'] = loans_df['employment_length'].str.replace('years', '', regex=True)
loans_df['employment_length'] = loans_df['employment_length'].str.replace('year', '', regex=True)

In [ ]:
class DataFrameInfo:
    def __init__(self, loans_df):
        self.loans_df = loans_df
         
    def collumn_description(self):
        return loans_df.describe()
    
    def loan_df_distinct(self):
        return loans_df.nunique(axis=0)
    
    def loans_df_shape(self):
        return loans_df.shape
    
    def loans_df_null_count(self):
        return loans_df.isnull()
    
    def loans_df_null_count_percent(self):
        return loans_df.isnull().sum()/len(loans_df)


In [ ]:
info = DataFrameInfo(loans_df)

In [ ]:
info.loan_df_distinct()

In [ ]:
info.loans_df_shape()

In [71]:
info.loans_df_null_count()

,member_id,loan_amount,funded_amount,funded_amount_inv,term,int_rate,instalment,grade,sub_grade,employment_length,...,recoveries,collection_recovery_fee,last_payment_date,last_payment_amount,next_payment_date,last_credit_pull_date,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type
id,,,,,,,,,,,,,,,,,,,,,
38676116,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
38656203,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
38656154,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,True,False,False
38656128,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,True,False,False
38656121,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76597,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,True,True,False,False
142608,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,True,False,False
117045,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,False,False


In [72]:
info.loans_df_null_count_percent()

member_id                      0.000000
loan_amount                    0.000000
funded_amount                  0.055448
funded_amount_inv              0.000000
term                           0.087994
int_rate                       0.095314
instalment                     0.000000
grade                          0.000000
sub_grade                      0.000000
employment_length              0.039055
home_ownership                 0.000000
annual_inc                     0.000000
verification_status            0.000000
issue_date                     0.000000
loan_status                    0.000000
payment_plan                   0.000000
purpose                        0.000000
dti                            0.000000
delinq_2yrs                    0.000000
earliest_credit_line           0.000000
inq_last_6mths                 0.000000
mths_since_last_delinq         0.571666
mths_since_last_record         0.886025
open_accounts                  0.000000
total_accounts                 0.000000


Changes:
id and member id in weird format 
change decimals (all) 
change names to be clearer
identify what 0 mean 
only need member id other id was automaticcaly generated


In [ ]:
loans_df.info()

In [82]:
from scipy.stats import normaltest
from matplotlib import pyplot
class Plotter:
    def __init__(self, loans_df):
        self.loans_df = loans_df
    
    def null_percent_funded(self):
        return loans_df['funded_amount'].isnull().sum() * 100/len(loans_df['funded_amount'])
        loans_df['funded_amount'].hist(bins=40)
    
    def null_percent_term(self):
        return loans_df['term'].isnull().sum() * 100/len(loans_df['term'])
    
    def null_percent_int_rate(self):
        return loans_df['int_rate'].isnull().sum() * 100/len(loans_df['int_rate'])
    
    def null_percent_empl_rate(self):
        return loans_df['employment_rate'].isnull().sum() * 100/len(loans_df['employment_rate'])
    
plotted_info = Plotter(loans_df)
plotted_info.null_percent_funded()
plotted_info.null_percent_term()
plotted_info.null_percent_int_rate()
plotted_info.null_percent_empl_rate()

9.531448802345523

In [ ]:
loans_df['funded_amount'].hist(bins=40)
loans_df['term'].hist(bins=40)
loans_df['int_rate'].hist(bins=40)

In [83]:
class DataFrameTransform:
    def __init__(self, loans_df):
        self.loans_df = loans_df

    def remove_na_several_collumns(self):
        return loans_df.loc[:,loans_df.notna().all(axis=0)]
    
    def remove_empty_str(self):
        return loans_df.loc[:,(loans_df!='').all(axis=0)]
    
transform = DataFrameTransform(loans_df)
transform.remove_na_several_collumns()
transform.remove_empty_str()


,member_id,loan_amount,funded_amount,funded_amount_inv,term,int_rate,instalment,grade,sub_grade,employment_length,...,recoveries,collection_recovery_fee,last_payment_date,last_payment_amount,next_payment_date,last_credit_pull_date,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type
id,,,,,,,,,,,,,,,,,,,,,
38676116,41461848,8000,8000.0,8000.0,36,7.49,248.82,A,A4,5,...,0.00,0.00,Jan-2022,248.82,Feb-2022,Jan-2022,0.0,5.0,1,INDIVIDUAL
38656203,41440010,13200,13200.0,13200.0,36,6.99,407.52,A,A3,9,...,0.00,0.00,Jan-2022,407.52,Feb-2022,Jan-2022,0.0,NaN,1,INDIVIDUAL
38656154,41439961,16000,16000.0,16000.0,36,7.49,497.63,A,A4,8,...,0.00,0.00,Oct-2021,12850.16,NaN,Oct-2021,0.0,NaN,1,INDIVIDUAL
38656128,41439934,15000,15000.0,15000.0,36,14.31,514.93,C,C4,1,...,0.00,0.00,Jun-2021,13899.67,NaN,Jun-2021,0.0,NaN,1,INDIVIDUAL
38656121,41439927,15000,15000.0,15000.0,36,6.03,456.54,A,A1,10+,...,0.00,0.00,Jan-2022,456.54,Feb-2022,Jan-2022,0.0,NaN,1,INDIVIDUAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76597,76583,5000,5000.0,1775.0,36,9.01,159.03,B,B2,1,...,0.00,0.00,Jul-2016,160.61,NaN,Jul-2016,NaN,NaN,1,INDIVIDUAL
142608,74724,5000,5000.0,2350.0,36,10.59,162.73,C,C2,< 1,...,0.00,0.00,Oct-2016,490.01,NaN,Sep-2016,0.0,NaN,1,INDIVIDUAL
117045,70978,3500,3500.0,2225.0,36,7.43,108.77,A,A2,10+,...,0.00,0.00,Sep-2016,110.58,Sep-2016,May-2013,NaN,NaN,1,INDIVIDUAL
